In [1]:
# Basic NGSolve things
from netgen.geom2d import SplineGeometry
from ngsolve import *

# ngsxfem and the mlset convenience layer 
from xfem import *
from xfem.mlset import *

import matplotlib.pyplot as plt
import numpy as np

## 1) Mixed

In [4]:
# mixed
geo = SplineGeometry()
geo.AddRectangle((0, 0), (1, 1), 
                 bcs=("bottom", "right", "top", "left"), 
                leftdomain=1, rightdomain=0)
geo.AddCircle((0.5, 0.5), r=0.25, 
                 bc="frac",
             leftdomain=1,
             rightdomain=1)

mesh = Mesh(geo.GenerateMesh(maxh=0.025))



# hybrid-mixed solver: GOOD
order = 0
frac = "frac"
V = HDiv(mesh, order=order, RT=True, dirichlet="top|bottom")
W = L2(mesh, order=order)
fes = V*W
(u, p), (v, q) = fes.TnT()

eps = 1
K0, K1 = 1, 1e-2

condense=False
a = BilinearForm(fes, condense=condense)
h = specialcf.mesh_size
n = specialcf.normal(2)
t = specialcf.tangential(2)

a += (1/K0*u*v-p*div(v)-q*div(u))*dx
a += eps/K1*u.Trace()*n*v.Trace()*n*ds("frac")

########### TODO

f = LinearForm(fes)
f += -v.Trace()*n*ds("left")
gfu = GridFunction(fes)

a.Assemble()
f.Assemble()

f.vec.data -= a.mat*gfu.vec
if condense==True:
    f.vec.data += a.harmonic_extension_trans * f.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs(condense), inverse="pardiso")*f.vec
if condense==True:
    gfu.vec.data += a.harmonic_extension * gfu.vec 
    gfu.vec.data += a.inner_solve * f.vec

Draw(gfu.components[0], mesh, "vel")
Draw(gfu.components[1], mesh, "soln")

WebGuiWidget(value={'ngsolve_version': '6.2.2103-72-g646b7caba', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2103-72-g646b7caba', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

## 2) Hybrid-Mixed

In [5]:
geo = SplineGeometry()
geo.AddRectangle((0, 0), (1, 1), 
                 bcs=("bottom", "right", "top", "left"), 
                leftdomain=1, rightdomain=0)
geo.AddCircle((0.5, 0.5), r=0.25, 
                 bc="frac",
             leftdomain=1,
             rightdomain=1)

mesh = Mesh(geo.GenerateMesh(maxh=0.025))

### level set info
phi = sqrt((x-0.5)**2+(y-0.5)**2)-0.25
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(phi,lsetp1)
ci = CutInfo(mesh, lsetp1)
lset_if  = { "levelset" : lsetp1, "domain_type" : IF , "subdivlvl" : 0}
lset_elem = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(lset_elem), mesh)



# hybrid-mixed solver: GOOD
order = 0
frac = "frac"
V = HDiv(mesh, order=order, RT=True, discontinuous=True)
W = L2(mesh, order=order)
M = FacetFESpace(mesh, order=order, dirichlet="left|right")
fes = V*W*M
(u, p, phat), (v, q, qhat) = fes.TnT()

eps = 1
K0, K1 = 1, 1e-2

condense=True
a = BilinearForm(fes, condense=condense)
h = specialcf.mesh_size
n = specialcf.normal(2)
t = specialcf.tangential(2)

a += (1/K0*u*v-p*div(v)-q*div(u))*dx
# only evalued on one side
# a += eps/K1*u.Trace()*n*v.Trace()*n*ds("frac")

# level set contribution
n0  = 1.0/grad(lsetp1).Norm() * grad(lsetp1)
a += SymbolicBFI(levelset_domain = lset_if, 
                 form = eps/K1 * u*n0 * v*n0,
                definedonelements=lset_elem)
# a hacker
# gf0 = GridFunction(FacetFESpace(mesh))
# gf0.Set(1, definedon=mesh.Boundaries("frac"))
# a += gf0*eps/K1 * u*n * v*n*dx(element_boundary=True, definedonelements=lset_elem)


a += (phat*v*n+qhat*u*n)*dx(element_boundary=True)

########### TODO

f = LinearForm(fes)
# f += -v.Trace()*n*ds("left")
gfu = GridFunction(fes)

a.Assemble()
f.Assemble()

gfu.components[2].Set(1, definedon=mesh.Boundaries("left"))
f.vec.data -= a.mat*gfu.vec
if condense==True:
    f.vec.data += a.harmonic_extension_trans * f.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs(condense), inverse="pardiso")*f.vec
if condense==True:
    gfu.vec.data += a.harmonic_extension * gfu.vec 
    gfu.vec.data += a.inner_solve * f.vec

Draw(gfu.components[0], mesh, "vel")
Draw(gfu.components[1], mesh, "soln")

WebGuiWidget(value={'ngsolve_version': '6.2.2103-72-g646b7caba', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2103-72-g646b7caba', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene